In [1]:
import pandas as pd
import numpy as np

# Data Preparation 

**Goal:** gather the different data sources to get a dataset including around 10 features and at least 1000 entries

**Data Preparation tasks:**
- [x] Agreggation of data sources and cleaning of columns names
- [x] Merging data sources
- [ ] Checking missing values and interpolation
- [ ] Creating calculated columns
- [ ] Add web-scraping of doctolib (?)

In [21]:
# Reading the dataset
apl = pd.read_excel('raw_data/apl-drees.xlsx',sheet_name='APL_2018',header=7, index_col=None)
print("Shape:",apl.shape)
apl.head()

Shape: (34990, 5)


,Code commune INSEE,Communes,APL aux médecins généralistes,APL aux médecins généralistes de moins de 65 ans,Population standardisée 2016 pour la médecine générale
0,NaN,NaN,En nombre de consultations/visites accessibles...,En nombre de consultations/visites accessibles...,En nombre d'habitants standardisés
1,01001,L'Abergement-Clémenciat,2.396,2.112,761.728
2,01002,L'Abergement-de-Varey,2.721,2.634,241.621
3,01004,Ambérieu-en-Bugey,4.335,4.271,13798.6
4,01005,Ambérieux-en-Dombes,4.279,4.028,1634.55


In [22]:
# Dropping first row and last 2 columns that we won't use
apl.drop(0,axis=0, inplace=True)
apl.drop(apl.iloc[:,-2:],axis=1, inplace=True)

In [154]:
# Cleaning of columns names to make it easier to work
apl = apl.rename(columns={'Code commune INSEE':'CODGEO','APL aux médecins généralistes':'APL'})

# Converting dtype of CODGEO to ensure future merge
apl.CODGEO = apl.CODGEO.astype(str)
apl = apl.convert_dtypes()
apl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34989 entries, 1 to 34989
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CODGEO    34989 non-null  string 
 1   Communes  34989 non-null  string 
 2   APL       34989 non-null  float64
dtypes: float64(1), string(2)
memory usage: 1.1 MB


### Secondary dataset: communes comparateur

Features to get from this data source: 
- Population
- Density area (hab/Km2 - Population/Superficie)
- Population growth
- Mediane Niveau de vie
- Unemployement Rate
- Part of secondary residences (%)
- Part of vacant residences (%)
- Part of city amenities (commerces, services, transports - %)
- Part of city amenities (administration, social, health, education - %)

In [61]:
# Reading dataset
com = pd.read_excel('raw_data/base_cc_comparateur.xls',sheet_name='COM',header=5)
print("Shape:",com.shape)
com.head()

Shape: (34953, 36)


,CODGEO,LIBGEO,REG,DEP,P16_POP,P11_POP,SUPERF,NAIS1116,DECE1116,P16_MEN,...,P16_ACT1564,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015
0,01001,L'Abergement-Clémenciat,84,01,767,780,15.95,41,25,306.000000,...,376.000000,50.0,11.0,3.0,5.0,24.0,5.0,7.0,10.0,0.0
1,01002,L'Abergement-de-Varey,84,01,243,234,9.15,21,7,101.000000,...,123.000000,19.0,2.0,3.0,0.0,11.0,3.0,3.0,2.0,0.0
2,01004,Ambérieu-en-Bugey,84,01,14081,13839,24.60,1114,595,6348.757303,...,6697.333122,1337.0,7.0,52.0,131.0,907.0,290.0,240.0,399.0,109.0
3,01005,Ambérieux-en-Dombes,84,01,1671,1600,15.92,101,42,640.000000,...,864.845592,141.0,14.0,7.0,27.0,78.0,20.0,15.0,27.0,5.0
4,01006,Ambléon,84,01,110,112,5.88,9,6,53.000000,...,58.000000,7.0,0.0,0.0,0.0,5.0,1.0,2.0,0.0,0.0


Information on selected columns:
- P16_POP	    Population en 2016
- P11_POP	    Population en 2011
- SUPERF	    Superficie (en km2)
- NAIS1116	    Nombre de naissances entre le 01/01/2011 et le 01/01/2016
- P16_LOG	    Nombre de logements en 2016
- P16_RSECOCC	Rés secondaires et logts occasionnels en 2016
- P16_LOGVAC	Logements vacants en 2016
- MED16	        Médiane du niveau vie en 2016
- P16_POP1564	Nombre de personnes de 15 à 64 ans en 2016
- P16_CHOM1564	Nombre de chômeurs de 15 à 64 ans en 2016
- ETTOT15	    Total des établissements actifs au 31 décembre 2015
- ETGU15	    Établissements actifs du commerce, transports et services divers au 31/12/2015
- ETOQ15	    Ets actifs de l'administration publique au 31/12/2015

In [64]:
# Selecting the interesting columns
sub_com = com[['CODGEO','P16_POP','SUPERF','P11_POP','P16_CHOM1564','P16_POP1564','P16_RSECOCC','P16_LOG',
     'P16_LOGVAC','ETGU15','ETOQ15','ETTOT15','MED16','NAIS1116']]

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
sub_com = sub_com.convert_dtypes()
sub_com.info()

In [162]:
# Merging the dataframes
df_merged = pd.merge(apl,sub_com,'left', on='CODGEO')
print(df_merged.shape)
df_merged.head()

(34989, 16)


,CODGEO,Communes,APL,P16_POP,SUPERF,P11_POP,P16_CHOM1564,P16_POP1564,P16_RSECOCC,P16_LOG,P16_LOGVAC,ETGU15,ETOQ15,ETTOT15,MED16,NAIS1116
0,01001,L'Abergement-Clémenciat,2.396,767,15.95,780,33.000000,463.000000,16.000000,348.000000,26.000000,24,7,50,22679.000000,41
1,01002,L'Abergement-de-Varey,2.721,243,9.15,234,10.000000,144.000000,52.000000,169.000000,16.000000,11,3,19,24382.083333,21
2,01004,Ambérieu-en-Bugey,4.335,14081,24.60,13839,1079.621428,8968.158251,120.067029,7126.116028,657.291696,907,240,1337,19721.000000,1114
3,01005,Ambérieux-en-Dombes,4.279,1671,15.92,1600,68.015722,1071.339842,12.433083,686.624060,34.190977,78,15,141,23378.000000,101
4,01006,Ambléon,0.912,110,5.88,112,8.000000,72.000000,12.000000,74.000000,9.000000,5,2,7,NaN,9


### Secondary dataset: evolution structures

Features to get from this datasource: 
- Repartition of population age
- Mobility_rate (% of population located 1 year ago)
- Socio-Professional Category

In [69]:
# Reading dataset
evol = pd.read_csv('raw_data/base-cc-evol-struct-pop-2016-csv/base-cc-evol-struct-pop-2016.CSV',sep=';')
print("Shape:",evol.shape)
evol.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (34998, 209)


,CODGEO,P16_POP,P16_POP0014,P16_POP1529,P16_POP3044,P16_POP4559,P16_POP6074,P16_POP7589,P16_POP90P,P16_POPH,...,C11_POP2554_CS8,C11_POP55P,C11_POP55P_CS1,C11_POP55P_CS2,C11_POP55P_CS3,C11_POP55P_CS4,C11_POP55P_CS5,C11_POP55P_CS6,C11_POP55P_CS7,C11_POP55P_CS8
0,1001,767.0,161.000000,102.000000,132.000000,189.000000,125.000000,53.000000,5.000000,392.000000,...,0.000000,224.000000,0.0,0.000000,16.000000,4.000000,20.000000,12.000000,164.000000,8.000000
1,1002,243.0,55.000000,28.000000,70.000000,37.000000,34.000000,17.000000,2.000000,123.000000,...,8.000000,72.000000,0.0,0.000000,0.000000,0.000000,0.000000,8.000000,64.000000,0.000000
2,1004,14081.0,2791.294950,2892.682129,2749.298629,2511.528964,1960.657550,1027.804290,147.733487,6813.623449,...,514.592587,3622.056361,0.0,40.354913,78.925152,136.259277,149.815525,170.164200,2800.673852,245.863442
3,1005,1671.0,342.918984,257.492465,330.675662,386.691402,239.848234,102.969273,10.403981,842.093176,...,27.722772,388.118812,0.0,7.920792,7.920792,7.920792,7.920792,11.881188,312.871287,31.683168
4,1006,110.0,12.000000,16.000000,15.000000,29.000000,27.000000,10.000000,1.000000,62.000000,...,8.000000,52.000000,0.0,0.000000,0.000000,8.000000,8.000000,0.000000,36.000000,0.000000


Information on selected columns:
- P16_POP01P	Nombre de personnes de 1 an ou plus localisée 1 an auparavant en 2016
- P16_POP0014	Pop 0-14 ans en 2016	
- P16_POP1529	Pop 15-29 ans en 2016		
- P16_POP3044	Pop 30-44 ans en 2016		
- P16_POP4559	Pop 45-59 ans en 2016		
- P16_POP6074	Pop 60-74 ans en 2016	
- P16_POP7589	Pop 75-89 ans en 2016
- P16_POP90P	Pop 90 ans ou plus en 2016
- C16_POP15P	Pop 15 ans ou plus en 2016		
- C16_POP15P_CS1	Pop 15 ans ou plus Agriculteurs exploitants en 2016
- C16_POP15P_CS2	Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2016
- C16_POP15P_CS3	Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2016
- C16_POP15P_CS4	Pop 15 ans ou plus Prof. intermédiaires  en 2016
- C16_POP15P_CS5	Pop 15 ans ou plus Employés en 2016
- C16_POP15P_CS6	Pop 15 ans ou plus Ouvriers en 2016
- C16_POP15P_CS7	Pop 15 ans ou plus Retraités en 2016
- C16_POP15P_CS8	Pop 15 ans ou plus Autres sans activité professionnelle en 2016

In [87]:
# Selecting the interesting columns
sub_evol = evol[['CODGEO','P16_POP01P']+list(evol.columns[2:9])+list(evol.columns[51:60])]

# Correcting CODGEO that have only 4 number by adding 0 before
sub_evol.CODGEO = sub_evol.CODGEO.apply(lambda x: '0'+str(x) if len(str(x))==4 else x).astype(str).copy()

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
sub_evol = sub_evol.convert_dtypes()
sub_evol.info()

In [166]:
# Merging the dataframes
df_merged_2 = pd.merge(df_merged,sub_evol,'left', on='CODGEO')
print(df_merged_2.shape)
df_merged_2.head()

(34989, 33)


,CODGEO,Communes,APL,P16_POP,SUPERF,P11_POP,P16_CHOM1564,P16_POP1564,P16_RSECOCC,P16_LOG,...,P16_POP90P,C16_POP15P,C16_POP15P_CS1,C16_POP15P_CS2,C16_POP15P_CS3,C16_POP15P_CS4,C16_POP15P_CS5,C16_POP15P_CS6,C16_POP15P_CS7,C16_POP15P_CS8
0,01001,L'Abergement-Clémenciat,2.396,767,15.95,780,33.000000,463.000000,16.000000,348.000000,...,5.000000,605.000000,15.000000,20.000000,75.000000,95.000000,100.000000,125.000000,145.000000,30.000000
1,01002,L'Abergement-de-Varey,2.721,243,9.15,234,10.000000,144.000000,52.000000,169.000000,...,2.000000,195.000000,0.000000,20.000000,15.000000,25.000000,40.000000,10.000000,65.000000,20.000000
2,01004,Ambérieu-en-Bugey,4.335,14081,24.60,13839,1079.621428,8968.158251,120.067029,7126.116028,...,147.733487,11273.704757,2.804736,300.150468,782.328599,1940.196262,1830.925313,1797.133429,2789.120337,1831.045614
3,01005,Ambérieux-en-Dombes,4.279,1671,15.92,1600,68.015722,1071.339842,12.433083,686.624060,...,10.403981,1374.056524,5.194086,61.990298,108.503109,237.313443,247.598151,237.081832,321.931876,154.443729
4,01006,Ambléon,0.912,110,5.88,112,8.000000,72.000000,12.000000,74.000000,...,1.000000,90.000000,0.000000,0.000000,5.000000,25.000000,15.000000,15.000000,25.000000,5.000000


### Secondary dataset: equipements

Features to get from this datasource: 
- Level of medical education

(other possible features: Number of leisure establishments, Number of healthcare establishments)

In [174]:
# Reading the dataset
eqmt = pd.read_csv('raw_data/bpe18_ensemble_csv/bpe18_ensemble.csv',sep=';')
print("Shape:",eqmt.shape)
eqmt.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (1035564, 7)


,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,NB_EQUIP
0,84,1,1001,01001,2018,A401,2
1,84,1,1001,01001,2018,A404,4
2,84,1,1001,01001,2018,A504,1
3,84,1,1001,01001,2018,A507,1
4,84,1,1001,01001,2018,B203,1


In [175]:
# Aggregating number of health education establishements
education_health = eqmt[eqmt.TYPEQU=='C402'].groupby('DEPCOM').NB_EQUIP.agg('sum').reset_index()

# Renaming DEPCOM as CODGEO to match with merge dataframe
education_health = education_health.rename(columns={'DEPCOM':'CODGEO'})

# Cleaning codes to add 0 when code is only 4 number
education_health.CODGEO = education_health.CODGEO.apply(lambda x: '0'+str(x) if len(str(x))==4 else x).astype(str).copy()

In [176]:
education_health = education_health.convert_dtypes()
education_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODGEO    524 non-null    string
 1   NB_EQUIP  524 non-null    Int64 
dtypes: Int64(1), string(1)
memory usage: 8.8 KB


In [177]:
df_merged_3 = pd.merge(df_merged_2, education_health,'left', on='CODGEO')
print(df_merged_3.shape)
df_merged_3.head()

(34989, 34)


,CODGEO,Communes,APL,P16_POP,SUPERF,P11_POP,P16_CHOM1564,P16_POP1564,P16_RSECOCC,P16_LOG,...,C16_POP15P,C16_POP15P_CS1,C16_POP15P_CS2,C16_POP15P_CS3,C16_POP15P_CS4,C16_POP15P_CS5,C16_POP15P_CS6,C16_POP15P_CS7,C16_POP15P_CS8,NB_EQUIP
0,01001,L'Abergement-Clémenciat,2.396,767,15.95,780,33.000000,463.000000,16.000000,348.000000,...,605.000000,15.000000,20.000000,75.000000,95.000000,100.000000,125.000000,145.000000,30.000000,<NA>
1,01002,L'Abergement-de-Varey,2.721,243,9.15,234,10.000000,144.000000,52.000000,169.000000,...,195.000000,0.000000,20.000000,15.000000,25.000000,40.000000,10.000000,65.000000,20.000000,<NA>
2,01004,Ambérieu-en-Bugey,4.335,14081,24.60,13839,1079.621428,8968.158251,120.067029,7126.116028,...,11273.704757,2.804736,300.150468,782.328599,1940.196262,1830.925313,1797.133429,2789.120337,1831.045614,<NA>
3,01005,Ambérieux-en-Dombes,4.279,1671,15.92,1600,68.015722,1071.339842,12.433083,686.624060,...,1374.056524,5.194086,61.990298,108.503109,237.313443,247.598151,237.081832,321.931876,154.443729,<NA>
4,01006,Ambléon,0.912,110,5.88,112,8.000000,72.000000,12.000000,74.000000,...,90.000000,0.000000,0.000000,5.000000,25.000000,15.000000,15.000000,25.000000,5.000000,<NA>


In [178]:
df_merged_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34989 entries, 0 to 34988
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CODGEO          34989 non-null  string 
 1   Communes        34989 non-null  string 
 2   APL             34989 non-null  float64
 3   P16_POP         34944 non-null  Int64  
 4   SUPERF          34944 non-null  float64
 5   P11_POP         34944 non-null  Int64  
 6   P16_CHOM1564    34944 non-null  float64
 7   P16_POP1564     34944 non-null  float64
 8   P16_RSECOCC     34944 non-null  float64
 9   P16_LOG         34944 non-null  float64
 10  P16_LOGVAC      34944 non-null  float64
 11  ETGU15          34943 non-null  Int64  
 12  ETOQ15          34943 non-null  Int64  
 13  ETTOT15         34943 non-null  Int64  
 14  MED16           31357 non-null  float64
 15  NAIS1116        34944 non-null  Int64  
 16  P16_POP01P      34989 non-null  float64
 17  P16_POP0014     34989 non-null 

In [181]:
# Filling nan values of Number of health education establishments because it means there isn't any.
df_merged_3.NB_EQUIP = df_merged_3.NB_EQUIP.fillna(0)

In [182]:
df_merged_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34989 entries, 0 to 34988
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CODGEO          34989 non-null  string 
 1   Communes        34989 non-null  string 
 2   APL             34989 non-null  float64
 3   P16_POP         34944 non-null  Int64  
 4   SUPERF          34944 non-null  float64
 5   P11_POP         34944 non-null  Int64  
 6   P16_CHOM1564    34944 non-null  float64
 7   P16_POP1564     34944 non-null  float64
 8   P16_RSECOCC     34944 non-null  float64
 9   P16_LOG         34944 non-null  float64
 10  P16_LOGVAC      34944 non-null  float64
 11  ETGU15          34943 non-null  Int64  
 12  ETOQ15          34943 non-null  Int64  
 13  ETTOT15         34943 non-null  Int64  
 14  MED16           31357 non-null  float64
 15  NAIS1116        34944 non-null  Int64  
 16  P16_POP01P      34989 non-null  float64
 17  P16_POP0014     34989 non-null 

In [183]:
# Saving the dataframe as it is if I want to retreive raw data
df_merged_3.to_csv('data/medical_desert_raw_data.csv',index=False)

________________________
## Cleaning missing values

In [185]:
df_merged_3.isna().sum()

CODGEO               0
Communes             0
APL                  0
P16_POP             45
SUPERF              45
P11_POP             45
P16_CHOM1564        45
P16_POP1564         45
P16_RSECOCC         45
P16_LOG             45
P16_LOGVAC          45
ETGU15              46
ETOQ15              46
ETTOT15             46
MED16             3632
NAIS1116            45
P16_POP01P           0
P16_POP0014          0
P16_POP1529          0
P16_POP3044          0
P16_POP4559          0
P16_POP6074          0
P16_POP7589          0
P16_POP90P           0
C16_POP15P           0
C16_POP15P_CS1       0
C16_POP15P_CS2       0
C16_POP15P_CS3       0
C16_POP15P_CS4       0
C16_POP15P_CS5       0
C16_POP15P_CS6       0
C16_POP15P_CS7       0
C16_POP15P_CS8       0
NB_EQUIP             0
dtype: int64

______________________________________
## Calculate new columns

The objective is to create calculated columns based on the data retrieved from the different sources. It will allow to add relative type of data (percentage) to make them more relevant to compare. 

In [59]:
# Calculated metrics
com['density_area'] = com.P16_POP / com.SUPERF
com['annual_pop_growth'] = ((com.P16_POP/com.P11_POP)**(1/(2016-2011))-1)*100
com['unemployment_rate'] = (com.P16_CHOM1564/com.P16_POP1564)*100
com['secondary_residence_rate'] = (com.P16_RSECOCC/com.P16_LOG)*100
com['vacant_residence_rate'] = (com.P16_LOGVAC/com.P16_LOG)*100
com['active_local_business_rate'] = (com.ETGU15/com.ETTOT15)*100
com['city_social_amenities_rate'] = (com.ETOQ15/com.ETTOT15)*100
com = com.rename(columns={'MED16':'median_living_standard'})

com.head()

,CODGEO,LIBGEO,REG,DEP,P16_POP,P11_POP,SUPERF,NAIS1116,DECE1116,P16_MEN,...,ETOQ15,ETTEF115,ETTEFP1015,density_area,annual_pop_growth,unemployment_rate,secondary_residence_rate,vacant_residence_rate,active_local_business_rate,city_social_amenities_rate
0,01001,L'Abergement-Clémenciat,84,01,767,780,15.95,41,25,306.000000,...,7.0,10.0,0.0,48.087774,-0.335578,7.127430,4.597701,7.471264,48.000000,14.000000
1,01002,L'Abergement-de-Varey,84,01,243,234,9.15,21,7,101.000000,...,3.0,2.0,0.0,26.557377,0.757662,6.944444,30.769231,9.467456,57.894737,15.789474
2,01004,Ambérieu-en-Bugey,84,01,14081,13839,24.60,1114,595,6348.757303,...,240.0,399.0,109.0,572.398374,0.347315,12.038385,1.684887,9.223702,67.838444,17.950636
3,01005,Ambérieux-en-Dombes,84,01,1671,1600,15.92,101,42,640.000000,...,15.0,27.0,5.0,104.962312,0.872154,6.348660,1.810755,4.979578,55.319149,10.638298
4,01006,Ambléon,84,01,110,112,5.88,9,6,53.000000,...,2.0,0.0,0.0,18.707483,-0.359722,11.111111,16.216216,12.162162,71.428571,28.571429
